# MIMIC-III tables to FHIR resource mapping in this notebook

||Original format | FHIR resource| Progress|Final Check|
|------|:-----|:-----|:-----|---:---|
|1|patients | patient| C,L,A|Done|
|2a|admissions | encounter| C,L,A|Done|
|2b|diagnoses_icd | encounter| C,L,A|Done|
|3|icustay | enounter|C,L,A|Done|
|4|cptevents | claim| C,L,A|Done|
|5|noteevents | diagnosticReport| C,L,A|Done|
<!-- |6|inputevents_cv | medicationDispense| C,L,A|Done|
|7|inputevents_mv | medicationDispense| C,L,A|Done|
|8|prescriptions | medicationRequest| C,L,A|Done|
|9|chartevents | observation| C,L,A|Done|
|10|datetimeevents | observation| C,L,A|Done|
|11|labevents | observation| C,L,A|Done|
|12|caregivers | practitioner| C,L,A|Done|
|13|procedures_icd | procedure| C,L,A|Done|
|14|procedureevents_mv | procedure| C,L,A|Done|
|15|microbiology | specimen| C,L,A|Done|
|16|outputevents | specimen| C,L,A|Done|
|17|service | serviceRequest|C,L,A|Done|
|18|callout | -|-|-|
|19|transfers | -|-|-|
|20|drgcodes | -|-|-| -->

## Import packages and data

In [1]:
import numpy as np
import pandas as pd
import os
import gc

In [2]:
data_path = './data/mimic-iii-clinical-database-1.4/'
output_path = './data/fhir_out/'
file_ext = '.csv.gz'
compression = 'gzip'

data_files = os.listdir(data_path)

In [22]:
# https://github.com/pandas-dev/pandas/issues/33196
# https://github.com/pandas-dev/pandas/pull/33398
# pd.options.io.compressionlevel = 1

# Create each FHIR resource type table individually

## fhir.patients table

#### MAPPING:<br>

- As soon as this table is joineed with another table the identifier column needs to be renamed to 'subject'
- We could add the deathtime from the admissions table, which could be more accurate.
- Check https://www.hl7.org/fhir/valueset-languages.html for language codes(probably easiest to just create a bigger version defining the language the language name is in and the language name itself)
- Using MARITIAL_STATUS from last encounter

||Original format | FHIR resource format|
|------|:-----|:-----|
|1|mimic.patients.SUBJECT_ID | fhir.patient.identifier|
|2|mimic.patients.GENDER | fhir.patient.gender|
|3|mimic.patients.DOB | fhir.patient.birthDate|
|4|mimic.patients.DOD | fhir.patient.deceasedDateTime|
|5|mimic.admissions.LANGUAGE | fhir.patient.communication_language|
|6|mimic.admissions.MARITIAL_STATUS | fhir.patient.maritialStatus|

In [3]:
def transform_patients(data_path, output_path):
    patients = pd.read_csv(data_path+'PATIENTS'+file_ext, compression=compression,
                           usecols=['SUBJECT_ID', 'GENDER', 'DOB', 'DOD'],
                           dtype={'SUBJECT_ID': int, 'GENDER': 'category'}, 
                           parse_dates=['DOB', 'DOD'])
    
    admissions = pd.read_csv(data_path+'ADMISSIONS'+file_ext, compression=compression,
                             usecols=['SUBJECT_ID', 'LANGUAGE', 'MARITAL_STATUS'],
                             dtype={'SUBJECT_ID': int, 'LANGUAGE': 'category', 'MARITAL_STATUS': 'category'})
    
    # Add the marital status and language information from the admissions dataframe by using the information from the last admission.
    ad_temp = admissions.groupby('SUBJECT_ID').tail(1)
    patients = pd.merge(patients, ad_temp, on='SUBJECT_ID')

    # Rename columns to FHIR names
    patients.rename(columns={'SUBJECT_ID':'identifier',
                             'GENDER':'gender', 
                             'DOB':'birthDate', 
                             'DOD':'deceasedDateTime',
                             'LANGUAGE':'communication_language',
                             'MARITAL_STATUS':'maritalStatus'}, inplace=True)

    # Output to csv
    patients.to_csv(output_path+'patient.csv.gz', compression='gzip', index=False)
    return patients

patient_fhir = transform_patients(data_path, output_path)
patient_fhir.head()

,identifier,gender,birthDate,deceasedDateTime,communication_language,maritalStatus
0,249,F,2075-03-13,NaT,ENGL,DIVORCED
1,250,F,2164-12-27,2188-11-22,HAIT,SINGLE
2,251,M,2090-03-15,NaT,NaN,NaN
3,252,M,2078-03-06,NaT,NaN,SINGLE
4,253,F,2089-11-26,NaT,NaN,WIDOWED


In [4]:
del patient_fhir
gc.collect()

17

## fhir.encounter table

#### ADMISSIONS MAPPING:<br>

- location column (add all interim locations and periods)
- - Could add location field by using Transfer and SERVICES mimictables
- - Combine both mimic.admissions.EDREGTIME and mimic.admissions.EDOUTTIME to add time period for fhir.encounter.locations list
- mimic.admissions.INSURANCE - Might be better placed in claims fhir table

||Original format | FHIR resource format|
|------|:-----|:-----|
|1|mimic.admissions.SUBJECT_ID|fhir.encounter.subject|
|2|mimic.admissions.HADM_ID|fhir.encounter.identifier|
|3|'admission'|fhir.encounter.serviceType|
|4|mimic.admissions.ADMITTIME|fhir.encounter.period_start|
|5|mimic.admissions.DISCHTIME|fhir.encounter.period_end|
|6|mimic.admissions.(DISCHTIME - ADMITTIME)|fhir.encounter.length|
|7|mimic.admissions.ADMISSION_TYPE|fhir.encounter.type|
|8|mimic.admissions.ADMISSION_LOCATION|fhir.encounter.hospitalization_origin|
|9|mimic.admissions.DISCHARGE_LOCATION|fhir.encounter.hospitalization_destination|
|10|mimic.admissions.INSURANCE|fhir.encounter.serviceProvider|
|11|mimic.DIAGNOSES_ICD.ICD9_CODE|fhir.encounter.diagnosis_condition|
|12|mimic.DIAGNOSES_ICD.SEQ_NUM|fhir.encounter.diagnosis_condition|
|13|mimic.admissions.DIAGNOSIS|fhir.encounter.diagnosis_description|

#### ICUSTAYS MAPPING:<br>

- Could improve location field by using Transfer table

||Original format | FHIR resource format|
|------|:-----|:-----|
|1|mimic.icustays.SUBJECT_ID|fhir.encounter.subject|
|2|mimic.icustays.HADM_ID|fhir.encounter.partOf|
|3|mimic.icustays.ICUSTAY_ID|fhir.encounter.identifier|
|4|'icustay'|fhir.encounter.serviceType|
|5|mimic.icustays.INTIME|fhir.encounter.period_start|
|6|mimic.icustays.OUTTIME|fhir.encounter.period_end|
|7|mimic.icustays.(OUTTIME - INTIME)|fhir.encounter.length|
|8|mimic.icustays.DBSOURCE|fhir.encounter.type|
|9|mimic.icustays.FIRST_CAREUNIT|fhir.encounter.location|
|10|mimic.icustays.LAST_CAREUNIT|fhir.encounter.location|
|11|mimic.icustays.FIRST_WARDID|fhir.encounter.location_id|
|12|mimic.icustays.LAST_WARDID|fhir.encounter.location_id|

In [5]:
def transform_admissions(data_path, output_path):
    admissions = pd.read_csv(data_path+'ADMISSIONS'+file_ext, compression=compression,
                             usecols=['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
                                      'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
                                      'INSURANCE', 'DIAGNOSIS'],
                             dtype={'SUBJECT_ID': int, 'HADM_ID': int, 'ADMISSION_TYPE': 'category',
                                    'ADMISSION_LOCATION': 'category', 'DISCHARGE_LOCATION': 'category', 
                                    'INSURANCE': 'category', 'DIAGNOSIS': str},
                             parse_dates=['ADMITTIME', 'DISCHTIME'])
    
    diagnoses_icd = pd.read_csv(data_path+'DIAGNOSES_ICD'+file_ext, compression=compression,
                                usecols=['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'],
                                dtype={'SUBJECT_ID': int, 'HADM_ID': int, 'SEQ_NUM': float, 'ICD9_CODE': str})
    
    # Convert times to datetime formats and add length column
    #admissions.ADMITTIME = pd.to_datetime(admissions.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    #admissions.DISCHTIME = pd.to_datetime(admissions.DISCHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    admissions['length'] = admissions['DISCHTIME'] - admissions['ADMITTIME']

    # Add diagnoses icd list. Icd codes automatically in the order of priority (descending priority), which is important for billing.
    diagnoses_icd_list = diagnoses_icd.groupby('HADM_ID')['ICD9_CODE'].apply(list).reset_index(name='diagnosis_condition')
    admissions = pd.merge(admissions, diagnoses_icd_list, on='HADM_ID')
    
    # Define encounter service type
    admissions['serviceType'] = 'admission'
    
    # Rename columns to FHIR names
    admissions.rename(columns={'SUBJECT_ID':'subject',
                               'HADM_ID':'identifier', 
                               'ADMITTIME':'period_start', 
                               'DISCHTIME':'period_end', 
                               'ADMISSION_TYPE':'type',
                               'ADMISSION_LOCATION':'hospitalization_origin',
                               'DISCHARGE_LOCATION':'hospitalization_destination',
                               'INSURANCE':'serviceProvider',
                               'DIAGNOSIS':'diagnosis_description'}, inplace=True)
    
    admissions = admissions.reindex(columns=['subject', 
                                             'identifier', 
                                             'period_start', 
                                             'period_end',
                                             'length',
                                             'type',
                                             'hospitalization_origin',
                                             'hospitalization_destination',
                                             'serviceProvider',
                                             'diagnosis_condition',
                                             'diagnosis_description'], copy=False)
    
    # Output to csv
    admissions.to_csv(output_path+'encounter.csv.gz', compression='gzip', index=False)
    return admissions

encounter_admissions = transform_admissions(data_path, output_path)
encounter_admissions.head()

,subject,identifier,period_start,period_end,length,type,hospitalization_origin,hospitalization_destination,serviceProvider,diagnosis_condition,diagnosis_description
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,1 days 03:28:00,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,"[9678, 9693, E9502, E9503, 3488, 29620, 4019]",BENZODIAZEPINE OVERDOSE
1,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,5 days 11:55:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,"[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...",CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...
2,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,6 days 18:26:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,"[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...",BRAIN MASS
3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,2 days 20:34:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,"[41041, 41401, 53081, 25000]",INTERIOR MYOCARDIAL INFARCTION
4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,3 days 12:49:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,"[41071, 25011, 41401, 4019]",ACUTE CORONARY SYNDROME


In [6]:
del encounter_admissions
gc.collect()

17

In [7]:
def transform_icustays(data_path, output_path):
    icustays = pd.read_csv(data_path+'ICUSTAYS'+file_ext, compression=compression,
                           usecols=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'DBSOURCE',
                                    'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'FIRST_WARDID',
                                    'LAST_WARDID', 'INTIME', 'OUTTIME', 'LOS'],
                           parse_dates=['INTIME', 'OUTTIME'],
                           dtype={'SUBJECT_ID': int, 'HADM_ID': int, 'ICUSTAY_ID': int,
                                  'DBSOURCE': 'category', 'FIRST_CAREUNIT': 'category',
                                  'LAST_CAREUNIT': 'category', 'FIRST_WARDID': int,
                                  'LAST_WARDID': int, 'LOS': float})    

    ##### !!! WHY, whe have 'LOS' columns for that !!!
    # Convert times to datetime formats and add length column
    #icustays.INTIME = pd.to_datetime(icustays.INTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    #icustays.OUTTIME = pd.to_datetime(icustays.OUTTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    #icustays['length'] = icustays['OUTTIME'] - icustays['INTIME']
    icustays['length'] = pd.to_timedelta(icustays['LOS'], unit='day')

    # Combine ADMISSION_LOCATION and DISCHARGE_LOCATION to location list
    icustays['location'] = icustays[['FIRST_CAREUNIT', 'LAST_CAREUNIT']].values.tolist()
    icustays.drop(['FIRST_CAREUNIT', 'LAST_CAREUNIT'], axis=1, inplace=True)
    icustays['location_id'] = icustays[['FIRST_WARDID', 'LAST_WARDID']].values.tolist()
    icustays.drop(['FIRST_WARDID', 'LAST_WARDID'], axis=1, inplace=True)
    
    # Define encounter service type
    icustays['serviceType'] = 'icustay'

    # Rename columns to FHIR names
    icustays.rename(columns={'SUBJECT_ID':'subject',
                             'HADM_ID':'partOf', 
                             'ICUSTAY_ID':'identifier',
                             'DBSOURCE':'type',
                             'INTIME':'period_start', 
                             'OUTTIME':'period_end'}, inplace=True)
    
    icustays = icustays.reindex(columns=['subject', 
                                         'partOf',
                                         'identifier', 
                                         'period_start', 
                                         'period_end',
                                         'length',
                                         'type',
                                         'serviceType',
                                         'location',
                                         'location_id'], copy=False)
    
    # Output to csv
    icustays.to_csv(output_path+'encounter_icustays.csv.gz', compression='gzip', index=False)
    return icustays

encounter_icustays = transform_icustays(data_path, output_path)
encounter_icustays.head()

,subject,partOf,identifier,period_start,period_end,length,type,serviceType,location,location_id
0,268,110404,280836,2198-02-14 23:27:38,2198-02-18 05:26:11,3 days 05:58:33.600000,carevue,icustay,"[MICU, MICU]","[52, 52]"
1,269,106296,206613,2170-11-05 11:05:29,2170-11-08 17:46:57,3 days 06:41:28.320000,carevue,icustay,"[MICU, MICU]","[52, 52]"
2,270,188028,220345,2128-06-24 15:05:20,2128-06-27 12:32:29,2 days 21:27:12.960000,carevue,icustay,"[CCU, CCU]","[57, 57]"
3,271,173727,249196,2120-08-07 23:12:42,2120-08-10 00:39:04,2 days 01:26:24,carevue,icustay,"[MICU, SICU]","[52, 23]"
4,272,164716,210407,2186-12-25 21:08:04,2186-12-27 12:01:13,1 days 14:53:05.280000,carevue,icustay,"[CCU, CCU]","[57, 57]"


In [8]:
del encounter_icustays
gc.collect()

17

## fhir.claim table

#### MAPPING: <br>

||Original format | FHIR resource format|
|------|:-----|:-----|
|1|mimic.cptevents.ROW_ID|fhir.claim.identifier|
|2|mimic.cptevents.SUBJECT_ID|fhir.claim.subject|
|3|mimic.cptevents.HADM_ID|fhir.claim.encounter|
|4|mimic.cptevents.COSTCENTER|fhir.claim.provider|
|5|mimic.cptevents.CHARTDATE|fhir.claim.created|
|6|mimic.cptevents.CPT_NUMBER|fhir.claim.item_category_cpt_num|
|7|mimic.cptevents.CPT_SUFFIX|fhir.claim.item_category_cpt_str|
|8|mimic.cptevents.TICKET_ID_SEQ|fhir.claim.item_sequence|
|9|mimic.cptevents.DESCRIPTION|fhir.claim.item_detail|
|10|mimic.cptevents.SECTIONHEADER|fhir.claim.type|
|11|mimic.cptevents.SUBSECTIONHEADER|fhir.claim.subType|


In [9]:
def transform_cptevents(data_path, output_path):
    cptevents = pd.read_csv(data_path+'CPTEVENTS'+file_ext, compression=compression,
                            usecols=lambda x: x not in ['CPT_CD'],
                            dtype={'ROW_ID': int, 'SUBJECT_ID': int, 'HADM_ID': int,  
                                   'SECTIONHEADER': 'category', 'SUBSECTIONHEADER': 'category',
                                   'COSTCENTER': 'category', 'CPT_NUMBER': float,
                                   'CPT_SUFFIX': str, 'TICKET_ID_SEQ': float,
                                   'DESCRIPTION': str},
                           parse_dates=['CHARTDATE'])
    
#     cptevents.CHARTDATE = pd.to_datetime(cptevents.CHARTDATE, format = '%Y-%m-%d', errors = 'coerce')
# 
#     Drop extra columns
#     cptevents.drop(['CPT_CD'], axis=1, inplace=True)

    # ROW_ID as identifier looks like a bad idea
    cptevents.rename(columns={'ROW_ID':'identifier',
                              'SUBJECT_ID':'subject', 
                              'HADM_ID':'encounter', 
                              'CHARTDATE':'created', 
                              'SECTIONHEADER':'type',
                              'SUBSECTIONHEADER':'subType',
                              'COSTCENTER':'provider',
                              'CPT_NUMBER':'item_category_cpt_num',
                              'CPT_SUFFIX':'item_category_cpt_str',
                              'TICKET_ID_SEQ':'item_sequence',
                              'DESCRIPTION':'item_detail'}, inplace=True)

    cptevents.to_csv(output_path+'claim.csv.gz', compression='gzip', index=False)
    return cptevents

claims = transform_cptevents(data_path, output_path)
claims.head()

,identifier,subject,encounter,provider,created,item_category_cpt_num,item_category_cpt_str,item_sequence,type,subType,item_detail
0,317,11743,129545,ICU,NaT,99232.0,NaN,6.0,Evaluation and management,Hospital inpatient services,NaN
1,318,11743,129545,ICU,NaT,99232.0,NaN,7.0,Evaluation and management,Hospital inpatient services,NaN
2,319,11743,129545,ICU,NaT,99232.0,NaN,8.0,Evaluation and management,Hospital inpatient services,NaN
3,320,11743,129545,ICU,NaT,99232.0,NaN,9.0,Evaluation and management,Hospital inpatient services,NaN
4,321,6185,183725,ICU,NaT,99223.0,NaN,1.0,Evaluation and management,Hospital inpatient services,NaN


In [10]:
del claims
gc.collect()

17

## fhir.diagnosticReport table

#### MAPPING: <br>

- DESCRIPTION isn't a code but comes closest to the idea of a concept.

||Original format | FHIR resource format|
|------|:-----|:-----|
|1|mimic.notevents.ROW_ID|fhir.diagnosticReport.identifier|
|2|mimic.notevents.SUBJECT_ID|fhir.diagnosticReport.subject|
|3|mimic.notevents.HADM_ID|fhir.diagnosticReport.encounter|
|4|mimic.notevents.CHARTDATE|fhir.diagnosticReport.effectiveDateTime|
|5|mimic.notevents.CGID|fhir.diagnosticReport.performer|
|6|mimic.notevents.CATEGORY|fhir.diagnosticReport.category|
|7|mimic.notevents.DESCRIPTION|fhir.diagnosticReport.codeDisplay|
|8|mimic.notevents.TEXT|fhir.diagnosticReport.presentedForm|
|9|mimic.notevents.ISERROR| fhir.diagnosticReport.status_error|


In [12]:
def transform_noteevents(data_path, output_path):
    noteevents = pd.read_csv(data_path+'NOTEEVENTS'+file_ext, compression=compression,
                             usecols=lambda x: x not in ['CHARTTIME', 'STORETIME'],
                             parse_dates=['CHARTDATE'],
                             dtype={'ROW_ID': int, 'SUBJECT_ID': int, 'HADM_ID': float,
                                    'CGID': float, 'CATEGORY': 'category',
                                    'DESCRIPTION': str, 'TEXT': str, 'ISERROR': str}) 
#     noteevents.CHARTDATE = pd.to_datetime(noteevents.CHARTDATE, format = '%Y-%m-%d', errors = 'coerce')
# 
#     Drop extra columns
#     noteevents.drop(['CHARTTIME', 'STORETIME'], axis=1, inplace=True)
    
    # ROW_ID as identifier looks like a bad idea
    noteevents.rename(columns={'ROW_ID':'identifier',
                                'SUBJECT_ID':'subject',
                                'HADM_ID':'encounter',
                                'CHARTDATE':'effectiveDateTime',
                                'CGID':'performer',
                                'CATEGORY':'category',
                                'DESCRIPTION':'codeDisplay',
                                'TEXT':'presentedForm',
                                'ISERROR':'status_error'}, inplace=True)

#     noteevents.to_csv(output_path+'diagnosticReport.csv.gz', compression='gzip', index=False) 
    return noteevents

diagnosticReport = transform_noteevents(data_path, output_path)
diagnosticReport.head()

,identifier,subject,encounter,effectiveDateTime,category,codeDisplay,performer,status_error,presentedForm
0,174,22532,167853.0,2151-08-04,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


## Speed tests

In [28]:
%%timeit -n1 -r1
# that takes a lot of time
diagnosticReport.to_csv(output_path+'diagnosticReport.csv.gz', compression='gzip', index=False)

5min 22s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [33]:
# %%timeit -n1 -r1
# https://github.com/pandas-dev/pandas/issues/33196
# https://github.com/pandas-dev/pandas/pull/33398
diagnosticReport.to_csv(output_path+'diagnosticReport.csv.gz', compression={'method': 'gzip', 'compresslevel': 1},
                        index=False)

2min 11s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [30]:
%%timeit -n1 -r1
# https://github.com/pandas-dev/pandas/issues/33196
# https://github.com/pandas-dev/pandas/pull/33398
diagnosticReport.to_csv(output_path+'diagnosticReport_3.csv.gz', compression={'method': 'gzip', 'compresslevel': 1},
                        index=False, chunksize=10000)

2min 14s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [31]:
%%timeit -n1 -r1
# https://github.com/pandas-dev/pandas/issues/33196
# https://github.com/pandas-dev/pandas/pull/33398
diagnosticReport.to_csv(output_path+'diagnosticReport_4.csv.gz', compression={'method': 'gzip', 'compresslevel': 1},
                        index=False, chunksize=100000)

2min 15s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [32]:
%%timeit -n1 -r1
# https://github.com/pandas-dev/pandas/issues/33196
# https://github.com/pandas-dev/pandas/pull/33398
diagnosticReport.to_csv(output_path+'diagnosticReport_5.csv.gz', compression={'method': 'gzip', 'compresslevel': 1},
                        index=False, chunksize=1000000)

2min 11s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
del diagnosticReport
gc.collect()